In [1]:
import pandas as pd
import plotly.graph_objects as go

pd.set_option('display.max_colwidth', None)

In [2]:
file_path = r'C:\Users\RafaelFagundes\OneDrive - Emma Sleep GmbH\Desktop\DA Assignment_NEW (5) (2)\DA Assignment_NEW\data_set_da_test\data_set_da_test.csv'

df = pd.read_csv(file_path)

### Exploratory Data Analysis

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637238 entries, 0 to 637237
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   event_date  637238 non-null  object
 1   session     637238 non-null  object
 2   user        637238 non-null  object
 3   page_type   637238 non-null  object
 4   event_type  637238 non-null  object
 5   product     637238 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 29.2+ MB


In [4]:
df.head()

,event_date,session,user,page_type,event_type,product
0,2022-10-08 17:02:41,14274187577460658115s,2006979063809820329u,search_listing_page,page_view,0
1,2022-10-08 17:06:19,14274187577460658115s,2006979063809820329u,search_listing_page,page_view,0
2,2022-10-08 22:19:47,2704204808571844605s,2007646148110679693u,listing_page,page_view,0
3,2022-10-08 22:24:30,8970170322512311099s,11839491588321754710u,search_listing_page,page_view,0
4,2022-10-08 21:22:20,16223970371660715740s,11839887495958431209u,product_page,page_view,0


In [5]:
df['event_type'].value_counts()

event_type
page_view      612498
add_to_cart     15999
order            8741
Name: count, dtype: int64

In [6]:
df['page_type'].value_counts()

page_type
product_page           282950
listing_page           231789
search_listing_page    113758
order_page               8741
Name: count, dtype: int64

In [7]:
df['session'].nunique()

340443

In [8]:
df['user'].nunique()

288088

### Custom Purchase Funnel (User Journey Analysis)

In [9]:
df['page_event_type'] = df['page_type'] + " (" + df['event_type'] + ")"

In [10]:
df['page_event_type'].value_counts()

page_event_type
product_page (page_view)             270750
listing_page (page_view)             231263
search_listing_page (page_view)      110485
product_page (add_to_cart)            12200
order_page (order)                     8741
search_listing_page (add_to_cart)      3273
listing_page (add_to_cart)              526
Name: count, dtype: int64

In [11]:
df_funnel_order = df.groupby(['page_event_type'])['session'].nunique().reset_index().sort_values(by=['session'], ascending=False)

df_funnel_order

,page_event_type,session
1,listing_page (page_view),180929
4,product_page (page_view),162773
6,search_listing_page (page_view),32489
3,product_page (add_to_cart),8879
2,order_page (order),7637
5,search_listing_page (add_to_cart),1992
0,listing_page (add_to_cart),386


In [12]:
stages = [
    "listing_page (page_view)",
    "product_page (page_view)",
    "product_page (add_to_cart)",
    "order_page (order)",
]

values = [180929, 162773, 8879, 7637]

fig = go.Figure(
    go.Funnel(
        y=stages,
        x=values,
        textinfo="value+percent initial",
        textposition="inside",
    )
)

fig.update_layout(
    title="How Users Progress Through the Purchase Funnel",
    width=1000,
    height=500,
)
fig.show()

### How the Journey Varies by Entry Page


In [13]:
funnel = df.pivot_table(
    index="session",
    columns="event_type",
    values="event_date",
    aggfunc="count",
    fill_value=0,
).reset_index()

funnel["page_viewed"] = funnel["page_view"] > 0
funnel["added_to_cart"] = funnel["add_to_cart"] > 0
funnel["ordered"] = funnel["order"] > 0


df_sorted = df.sort_values(["session", "event_date"])


first_page_type = (
    df_sorted.groupby("session").first().reset_index()[["session", "page_type"]]
)


first_page_type.rename(columns={"page_type": "first_page_type"}, inplace=True)


funnel = funnel.merge(first_page_type, on="session", how="left")



sessions_by_first_page = (
    funnel.groupby("first_page_type")
    .agg({"page_viewed": "sum", "added_to_cart": "sum", "ordered": "sum"})
    .reset_index()
)


sessions_by_first_page["add_to_cart_rate_%"] = (
    sessions_by_first_page["added_to_cart"] / sessions_by_first_page["page_viewed"]
)
sessions_by_first_page["order_conversion_rate_%"] = (
    sessions_by_first_page["ordered"] / sessions_by_first_page["page_viewed"]
)


sessions_by_first_page["page_view_distribution_%"] = (
    sessions_by_first_page["page_viewed"] / sessions_by_first_page["page_viewed"].sum()
)


sessions_by_first_page["drop_off_rate_%"] = (
    1 - sessions_by_first_page["order_conversion_rate_%"]
)

sessions_by_first_page = sessions_by_first_page[
    sessions_by_first_page["first_page_type"] != "order_page"
]



sessions_by_first_page = sessions_by_first_page[
    [
        "first_page_type",
        "page_viewed",
        "added_to_cart",
        "ordered",
        "page_view_distribution_%",
        "add_to_cart_rate_%",
        "order_conversion_rate_%",
        "drop_off_rate_%",
    ]
]


sessions_by_first_page

,first_page_type,page_viewed,added_to_cart,ordered,page_view_distribution_%,add_to_cart_rate_%,order_conversion_rate_%,drop_off_rate_%
0,listing_page,176831,3053,2358,0.521095,0.017265,0.013335,0.986665
2,product_page,146864,6564,3341,0.432787,0.044694,0.022749,0.977251
3,search_listing_page,15283,1040,539,0.045037,0.068049,0.035268,0.964732


In [38]:
funnel

,session,add_to_cart,order,page_view,page_viewed,added_to_cart,ordered,first_page_type
0,10000000052138714153s,0,0,1,True,False,False,search_listing_page
1,10000003251787879077s,0,0,2,True,False,False,search_listing_page
2,10000028460721229362s,0,0,2,True,False,False,listing_page
3,10000145746398845278s,0,0,1,True,False,False,listing_page
4,1000015740551295390s,0,0,1,True,False,False,listing_page
...,...,...,...,...,...,...,...,...
340438,9999792208481548234s,0,0,1,True,False,False,product_page
340439,999979223279301185s,0,0,2,True,False,False,product_page
340440,9999886616992267320s,0,0,1,True,False,False,listing_page
340441,9999924110261829254s,0,0,1,True,False,False,listing_page


In [53]:
abnormal = funnel[
    (funnel["ordered"] > 0) & ((funnel["add_to_cart"] == 0) | (funnel["page_view"] == 0))
]

print(abnormal['page_viewed'].sum())
print(abnormal['added_to_cart'].sum())
print(abnormal['ordered'].sum())

2356
27
3415


In [37]:

df_sorted = df.sort_values(['session', 'event_date'])

session_paths = df_sorted.groupby('session')['page_event_type'].apply(list).reset_index()

session_paths['path'] = session_paths['page_event_type'].apply(lambda x: ' > '.join(x))

path_counts = session_paths['path'].value_counts().reset_index()

path_counts.columns = ['path', 'count']

path_counts.head(15)


,path,count
0,listing_page (page_view),138601
1,product_page (page_view),104762
2,product_page (page_view) > product_page (page_view),17680
3,listing_page (page_view) > listing_page (page_view),13561
4,product_page (page_view) > product_page (page_view) > product_page (page_view),6028
5,search_listing_page (page_view),5893
6,listing_page (page_view) > listing_page (page_view) > listing_page (page_view),2743
7,product_page (page_view) > product_page (page_view) > product_page (page_view) > product_page (page_view),2466
8,listing_page (page_view) > product_page (page_view),2277
9,search_listing_page (page_view) > search_listing_page (page_view),1891
